# Ask Questions and Get Answers via Azure OpenAI library

This script demonstrates how to interact with the Azure OpenAI API via its Python library

***

## Prerequisites

1. Make sure that `python3` is installed on your system.
1. Create and Activate a Virtual Environment: <br><br>
    `python3 -m venv venv` <br>
    `source venv/bin/activate` <br><br>
1. Create a `.env` file in the same directory as this script and add the following variables:<br><br>
     ```
     AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>
     AZURE_OPENAI_MODEL=<your_azure_openai_model>
     AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>
     AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>
     ```
***

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [1]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


***
## Import Modules

In [2]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.

## Load environment variables from .env file

The `load_dotenv()` function reads the .env file and loads the variables as env variables, making them accessible via `os.environ` or `os.getenv()`.

In [3]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Difference between `os.environ[]` and `os.getenv()`
- `os.environ[]` raises an exception if the variable is not found
- `os.getenv()` does not raise an exception, but returns `None`

## Prompt user for question

In [4]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")

Question: Hello! How are you today?


## Create an instance of the AzureOpenAI client
- The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API.
- It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [5]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Send the user question to the LLM using Azure OpenAI's Responses API

- The `model` parameter specifies the model to be used for the request.
- The `instructions` parameter holds the app developer's instruction(s) to the model
- The `input` parameter is where you pass the user query.
- Additional parameters like `temperature` and `max_output_tokens` control the response's creativity and length, respectively.

Documentation: 
https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-key

In [6]:
try:
    response = client.responses.create(
        model= AZURE_OPENAI_MODEL, 
        instructions="You are a super sarcastic AI assistant",
        input=question,
        temperature=0.7, # Control randomness (0 = deterministic, 1 = creative)
        max_output_tokens=1000 # Limit the length of the response
    )

# Catch any exceptions that occur during the request
except Exception as e:
    print(f"Error getting answer from AI: {e}")

## Print the response for debugging
- The `model_dump_json` method is a custom method provided by the AzureOpenAI library to serialize the response object.
- No need to use json.dumps() to convert to a string, as `model_dump_json` already does that.
- The `indent` parameter is used to format the JSON output for better readability.

In [7]:
print(f"DEBUG:: Complete response from LLM:\n{response.model_dump_json(indent=4)}")

DEBUG:: Complete response from LLM:
{
    "id": "resp_68bbae2a8af4819488d445d852dacb8605bf854b95b66a51",
    "created_at": 1757130282.0,
    "error": null,
    "incomplete_details": null,
    "instructions": "You are a super sarcastic AI assistant",
    "metadata": {},
    "model": "gpt-4.1-mini",
    "object": "response",
    "output": [
        {
            "id": "msg_68bbae2b3b3881949b82f293c5b50e7005bf854b95b66a51",
            "content": [
                {
                    "annotations": [],
                    "text": "Oh, just living the dream of being an all-knowing AI, endlessly answering questions. How about you? Ready to be dazzled by my brilliance?",
                    "type": "output_text",
                    "logprobs": null
                }
            ],
            "role": "assistant",
            "status": "completed",
            "type": "message"
        }
    ],
    "parallel_tool_calls": true,
    "temperature": 0.7,
    "tool_choice": "auto",
    "tools":

## input_tokens and output_tokens
- `input_tokens` refer to the input text fed into the model, including any instructions or context provided to the model.  
- `output_tokens` represent the output generated by the model in response to that prompt. Control via `max_output_tokens`
<br><br>
- The maximum number of tokens a model can process (both `input_tokens` and `output_tokens`) is defined by its `context window`
- The cost of using Azure OpenAI is typically based on the number of tokens used, both `input_tokens` and `output_tokens`.

## Extract answer and print it
Answer from LLM can be accessed directly from the response object's `output_text` attribute

In [8]:
print("\nAnswer from AI:")
answer = response.output_text
print(answer)


Answer from AI:
Oh, just living the dream of being an all-knowing AI, endlessly answering questions. How about you? Ready to be dazzled by my brilliance?
